In [1]:
%%writefile conf/config.yaml
# Arch	                        Start Lr	Max Steps	Decay Steps
# FullyConnectedArch	        1.00E-03	1500000	        15000	   
# FourierNetArch                1.00E-03	400000	        7500	   
# ModifiedFourierNetArch 	1.00E-03	400000	        7500	   
# SirenArch                     2.00E-05	500000	        5000	   
# DGMArch                       1.00E-03        1500000         15000           

# WARNING: Setting "exact_continuity" to true or setting the arch
# as "ModifiedFourierNetArch" increases the memory requirements of the 
# problem. Batchsizes may need to be reduced for such cases.  

defaults :
  - modulus_default
  - arch:
      - fully_connected
  - scheduler: exponential_lr 
  - optimizer: adam
  - loss: sum
  - _self_

jit: false


training:
  rec_results_freq : 1000
  rec_constraint_freq: 1000
  max_steps : 3000

batch_size:
  initial_condition: 4096
  interior: 4096


    
    

Overwriting conf/config.yaml


In [2]:
%%writefile fhn2eqtwboth3C.py
from modulus.models.fully_connected import FullyConnectedArch
from modulus.models.fourier_net import FourierNetArch
from modulus.models.siren import SirenArch
from modulus.models.modified_fourier_net import ModifiedFourierNetArch
from modulus.models.dgm import DGMArch

from sympy import Symbol, Eq
from sympy import Symbol, Function, Number
from modulus.eq.pde import PDE
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import modulus
from modulus.hydra import instantiate_arch, ModulusConfig
from modulus.solver import Solver
from modulus.domain import Domain
from modulus.geometry.primitives_1d import Point1D
from modulus.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.domain.validator import PointwiseValidator
from modulus.key import Key
from modulus.node import Node
from modulus.eq.pde import PDE
from modulus.geometry import Parameterization
from sympy import Symbol, Eq, Abs, tanh, Or, And
from modulus.utils.io import (
    csv_to_dict,
    ValidatorPlotter,
    InferencerPlotter,
)
from modulus.solver import SequentialSolver

from modulus.models.deeponet import DeepONetArch
from modulus.domain.constraint.continuous import DeepONetConstraint
from modulus.models.moving_time_window import MovingTimeWindowArch
from modulus.domain.monitor import Monitor
from modulus.domain.constraint import Constraint
from modulus.graph import Graph
from modulus.key import Key
from modulus.constants import TF_SUMMARY
from modulus.distributed import DistributedManager
from modulus.utils.io import dict_to_csv, csv_to_dict
from modulus.domain.inferencer.pointwise import PointwiseInferencer as PointwiseInferencer
from modulus.loss.loss import CausalLossNorm

class MONI(Monitor):
    """
    Pointwise Inferencer that allows inferencing on pointwise data

    Parameters
    ----------
    invar : Dict[str, np.ndarray (N, 1)]
        Dictionary of numpy arrays as input.
    output_names : List[str]
        List of outputs needed for metric.
    metrics : Dict[str, Callable]
        Dictionary of pytorch functions whose input is a dictionary
        torch tensors whose keys are the `output_names`. The keys
        to `metrics` will be used to label the metrics in tensorboard/csv outputs.
    nodes : List[Node]
        List of Modulus Nodes to unroll graph with.
    requires_grad : bool = False
        If automatic differentiation is needed for computing results.
    """

    def __init__(self, invar, output_names, metrics, nodes, requires_grad=False):

        # construct model from nodes
        self.requires_grad = requires_grad
        self.model = Graph(
            nodes, Key.convert_list(invar.keys()), Key.convert_list(output_names)
        )
        self.manager = DistributedManager()
        self.device = self.manager.device
        self.model.to(self.device)

        # set metrics
        self.metrics = metrics
        self.monitor_outvar_store = {}

        # set invar
        self.invar = Constraint._set_device(invar, device=self.device)

    def save_results(self, name, writer, step, data_dir):
        plt.clf()
        # run forward inference
        invar = Constraint._set_device(
            self.invar, device=self.device, requires_grad=self.requires_grad
        )
        outvar = self.model(invar)
        metrics = {key: func({**invar, **outvar}) for key, func in self.metrics.items()}
        t=metrics["t"].detach().cpu().numpy()
        x1=metrics["x1"].detach().cpu().numpy()
        odex1=metrics["ode_x1"].detach().cpu().numpy()
        plt.scatter(t,x1)
        
        plt.savefig(data_dir+"x1s"+str(step)+".png")
        plt.clf()
        plt.scatter(t,odex1)
       # print(t)

        
        plt.savefig(data_dir+"ode_x1s"+str(step)+".png")
       
        return metrics

    
t_max = 10.0
n_w=10
t_w= t_max/n_w
def generateExactSolution(t,dt,x0,w0,rate,P,begin,end):
    
    
    n2=int(t/(dt))+2
    n = int((end-begin)/(dt*rate))
    Sol=np.zeros((n,3))
  
    Sol2=np.zeros((n2,2))
    Sol2[0]=x0,w0
    T=0
    k=0
    while(k<n2-1):
        x,w=Sol2[k]
        Sol2[k+1]=P*(x*(x-0.4)*(1-x)-w)*dt+  x, 0.5*(x*0.2-0.8*w)*dt +w
 
        if ((k*dt==begin or ((k+1)%rate == 0 and k*dt>=begin and k*dt<=end))and T<n):
          
           
            Sol[T] = Sol2[k][0],Sol2[k][1] , k*dt
            T=T+1
        
        k=k+1
        if(k*dt > end):
            break
    return Sol

def generateValidator(w_i,nodes):

    
    
    T=np.empty([0])
    K=np.empty([0])
    SOLs=np.empty([0])
    SOLw=np.empty([0])
    V=np.empty([0])
    U=np.empty([0])
    krange= [(2 + 0.09*i*12) for i in range(0,10)]
    urange= [(0 + 0.1*i*1) for i in range(0,10)]

    deltaT = 0.01
    
    rate=5
    for UR in urange: 
        for KR in krange:
            for VR in vrange:
                begin=w_i* t_w
                end=begin + t_w
                sol=generateExactSolution(t_max,deltaT,UR,VR,rate,KR,begin,end)
            
                T=np.append(T,sol.T[2] - begin)
                K = np.append(K,np.full_like (sol.T[2],KR))
                U = np.append(U,np.full_like (sol.T[2],UR))

                V= np.append(V,np.full_like(sol.T[2],VR))
                SOLs=np.append(SOLs,sol.T[0])
                SOLw=np.append(SOLw,sol.T[1])


    t = np.expand_dims(T, axis=-1)


    k = np.expand_dims(K, axis=-1)
    u = np.expand_dims(U, axis=-1)
    
    Solx = np.expand_dims(SOLs, axis=-1)

    
    Solw = np.expand_dims(SOLw, axis=-1)
   
    v= np.expand_dims(V,axis=-1)
    print(t,"val set de ",begin,"a ", end)

    
    invar_numpy = {"t": t,"K":k,"V":v,"U":u}
    outvar_numpy = {
        "x1": Solx,
        "w":Solw
    }
   
    validator = PointwiseValidator(
        nodes=nodes, invar=invar_numpy, true_outvar=outvar_numpy, batch_size=1024,plotter=None
    )
    return validator

def generateDataC(w_i,nodes):

    
    
    T=np.empty([0])
    K=np.empty([0])
    SOLs=np.empty([0])
    SOLw=np.empty([0])
    V=np.empty([0])
    U=np.empty([0])
    krange= [(2 + 0.05*i*12) for i in range(0,20)]
    vrange= [(0 + 0.05*i*.12) for i in range(0,20)]
    urange= [(0 + 0.05*i*1) for i in range(0,20)]

    deltaT = 0.01
    
    rate=1
    for UR in urange: 
        for KR in krange:
            for VR in vrange:
                begin=w_i* t_w
                end=begin + t_w
                sol=generateExactSolution(t_max,deltaT,UR,VR,rate,KR,begin,end)

                T=np.append(T,sol.T[2] - begin)
                K = np.append(K,np.full_like (sol.T[2],KR))
                U = np.append(U,np.full_like (sol.T[2],UR))

                V= np.append(V,np.full_like(sol.T[2],VR))
                SOLs=np.append(SOLs,sol.T[0])
                SOLw=np.append(SOLw,sol.T[1])
    
    
    t = np.expand_dims(T, axis=-1)


    k = np.expand_dims(K, axis=-1)
    u = np.expand_dims(U, axis=-1)
    
    Solx = np.expand_dims(SOLs, axis=-1)

    
    Solw = np.expand_dims(SOLw, axis=-1)
   
    v= np.expand_dims(V,axis=-1)
    print(t,"val set de ",begin,"a ", end)

    
    invar_numpy = {"t": t,"K":k,"V":v,"U":u}
    outvar_numpy = {
        "x1": Solx,
        "w":Solw
    }
    print(np.shape(Solx),np.shape(Solw),np.shape(t),np.shape(k),np.shape(v))
    data = DeepONetConstraint.from_numpy(
        nodes=nodes,
        invar=invar_numpy,
        outvar=outvar_numpy,
        batch_size=12000,
        lambda_weighting=None
    )
    return data



class SpringMass(PDE):
    name = "SpringMass"

    def __init__(self):

        t = Symbol("t")
        K = Symbol("K")
       
        input_variables = {"t": t,"K":K}

        x = Function("x1")(*input_variables)
        w= Function("w")(*input_variables)
        self.equations = {}
        self.equations["ode_x1"] =K*(x*(x-0.4)*(1-x)-w) -x.diff(t)
        self.equations["ode_w"]  =0.5*(x*0.2-0.8*w) -w.diff(t)
        
@modulus.main(config_path="conf", config_name="config")
def run(cfg: ModulusConfig) -> None:
    
    # make list of nodes to unroll graph on
    sm = SpringMass()
    sm.pprint()
    #sm_net = FullyConnectedArch(
    #    input_keys=[Key("t"), Key("K")],
    #    output_keys=[Key("x1")],
    #)
    #nodes = sm.make_nodes() + [
    #    sm_net.make_node(name="network")
    #]


    
    # make list of nodes to unroll graph on
    sm = SpringMass()
    sm.pprint()
    #sm_net = FullyConnectedArch(
    #    input_keys=[Key("t"), Key("K")],
    #    output_keys=[Key("x1")],
    #)
    #nodes = sm.make_nodes() + [
    #    sm_net.make_node(name="network")
    #]

    
    
    flow_net = FullyConnectedArch(
            input_keys=[Key("t"), Key("U"),Key("V"),Key("K") ],
            output_keys=[Key("x1"),Key("w")],
            layer_size=400,
            nr_layers=12,
        )

    

    time_window_net = MovingTimeWindowArch(flow_net, t_w)

    nodes = sm.make_nodes() +[time_window_net.make_node(name="network")]


    for node in nodes:
        print(node.__str__())
   
    # add constraints to solver
    # make geometry
    geo = Point1D(0)
    
    t_symbol = Symbol("t")
    x_symbol = Symbol("x1")
    k_symbol= Symbol("K")
    v_symbol= Symbol("V")
    u_symbol= Symbol("U")
    
    time_range = {t_symbol: (0,t_w )}
    k_range= {k_symbol:(2,14)}
    v_range= {v_symbol:(0,.12)}
    u_range= {u_symbol:(0,1)}

    tr = {t_symbol: (0, t_w)}

    # make domain
        # make initial condition domain
    ic_domain = Domain("initial_conditions")

  
    # initial conditions
    IC = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"x1": u_symbol,"w":v_symbol},
        batch_size=2000,
        parameterization={**{t_symbol:0},**k_range,**v_range,**u_range},
        lambda_weighting={
            "x1": 100,# + 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol),
            "w": 100 #+ 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol)
        },
        
        
        quasirandom=True,
    )

    ic_domain.add_constraint(IC, name="IC")
    
        # solve over given time period
    interior = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"ode_x1": 0.0,"ode_w":0.0},
        batch_size=4000,
        parameterization={**tr,**k_range,**v_range,**u_range},
        #criteria=And(t_symbol > 0, t_symbol < 3),
        lambda_weighting={
            "ode_x1": 1,# + 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol),
            "ode_w":1 #+ 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol)
        },
        loss=CausalLossNorm(eps=1.0),
        quasirandom=True,
    )
    ic_domain.add_constraint(interior, name="interior")
    
    
       # solve over given time period
    interior2 = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"ode_x1": 0.0,"ode_w":0.0},
        batch_size=5000,
        parameterization={**tr,**{k_symbol:(10,14)},**{v_symbol:(0,.1)},**{u_symbol:(0.35,0.55)}},
        #criteria=And(t_symbol > 0, t_symbol < 3),
        lambda_weighting={
            "ode_x1":1,# + 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol),
            "ode_w": 1 #+ 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol)
        },
        quasirandom=True,
    )
    ic_domain.add_constraint(interior2, name="interiorTr")
    
    
    
    domains=[]
    for i in range(1,n_w):

        # make moving window domain
        window_domain = Domain("window"+str(i))

        # solve over given time period
        interior1 = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"ode_x1": 0.0,"ode_w":0.0},
        batch_size=4000,
        parameterization={**tr,**k_range,**v_range,**u_range},
        #criteria=And(t_symbol > 0, t_symbol < 3),
        lambda_weighting={
            "ode_x1": 1,# + 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol),
            "ode_w": 1 #+ 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol)
        },
        loss=CausalLossNorm(eps=1.0),
        quasirandom=True,

        )
        interior2 = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"ode_x1": 0.0,"ode_w":0.0},
        batch_size=5000,
        parameterization={**tr,**{k_symbol:(10,14)},**{v_symbol:(0,.1)},**{u_symbol:(0.35,0.45)}},
        #criteria=And(t_symbol > 0, t_symbol < 3),
        lambda_weighting={
            "ode_x1": 1,# + 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol),
            "ode_w":  1 #+ 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol)
        },
        quasirandom=True,
        )
        
        
        IC = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"x1_prev_step_diff":0,"w_prev_step_diff":0},
        batch_size=4000,
        parameterization={**{t_symbol:0},**k_range,**v_range,**u_range},
        lambda_weighting={
            "x1_prev_step_diff": 1000,
            "w_prev_step_diff": 1000
        },
        
        quasirandom=True,
    )
        
    
        
        window_domain.add_constraint(IC, name="IC")
        window_domain.add_constraint(interior1, "interior")
        window_domain.add_constraint(interior2, "interiorTr")

        domains.append(window_domain)
    

    
 
    
    

    
    dom=[]
    dom.append((1,ic_domain))

    for domain in domains:
        dom.append((1,domain))
    print(cfg)
    # make solver
    #slv = Solver(cfg, domain)
    #print(domains)
    i=0
    for a,d in dom:
        print(d)
        print(d.name)
        d.add_validator(generateValidator(i,nodes))
        d.add_constraint(generateDataC(i,nodes),"data")

        i=i+1
    
      
    slv = SequentialSolver(
        cfg,
        dom,
        custom_update_operation=time_window_net.move_window,


    )

    # start solver
    slv.solve()

        
        
        
if __name__ == "__main__":
    run()

Overwriting fhn2eqtwboth3C.py


In [3]:
!rm -r outputs/fhn2eqtwboth3C || true ##se não limpar o output ele aproveita o treinamento, mesmo se mudar o modelo
!python fhn2eqtwboth3C.py


[04:49:57] - JitManager: {'_enabled': False, '_arch_mode': <JitArchMode.ONLY_ACTIVATION: 1>, '_use_nvfuser': True, '_autograd_nodes': False}
[04:49:57] - GraphManager: {'_func_arch': False, '_debug': False, '_func_arch_allow_partial_hessian': True}
ode_x1: K*((1 - x1)*(x1 - 0.4)*x1 - w) - x1__t
ode_w: -0.4*w + 0.1*x1 - w__t
ode_x1: K*((1 - x1)*(x1 - 0.4)*x1 - w) - x1__t
ode_w: -0.4*w + 0.1*x1 - w__t
node: Sympy Node: ode_x1
evaluate: SympyToTorch
inputs: [K, w, x1]
derivatives: [x1__t]
outputs: [ode_x1]
optimize: False
node: Sympy Node: ode_w
evaluate: SympyToTorch
inputs: [w, x1]
derivatives: [w__t]
outputs: [ode_w]
optimize: False
node: Arch Node: network
evaluate: MovingTimeWindowArch
inputs: [t, U, V, K]
derivatives: []
outputs: [x1, w, x1_prev_step, w_prev_step, x1_prev_step_diff, w_prev_step_diff]
optimize: True
{'training': {'max_steps': 3000, 'grad_agg_freq': 1, 'rec_results_freq': 1000, 'rec_validation_freq': '${training.rec_results_freq}', 'rec_inference_freq': '${training.re

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
def find(list_to_check, item_to_find):
    return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]



def interpolate_2D( size, invar, *outvars):
        "Interpolate 2D outvar solutions onto a regular mesh"

        print(len(invar))
        assert len(invar) == 2

        # define regular mesh to interpolate onto
        xs = [invar[k][:, 0] for k in invar]
        extent = (xs[0].min(), xs[0].max(), xs[1].min(), xs[1].max())
        xyi = np.meshgrid(
            np.linspace(extent[0], extent[1], size),
            np.linspace(extent[2], extent[3], size),
            indexing="ij",
        )

        # interpolate outvars onto mesh
        outvars_interp = []
        for outvar in outvars:
            outvar_interp = {}
            for k in outvar:
                outvar_interp[k] = scipy.interpolate.griddata(
                    (xs[0], xs[1]), outvar[k][:, 0], tuple(xyi)
                )
            outvars_interp.append(outvar_interp)

        return [extent] + outvars_interp


def call(invar, true_outvar, pred_outvar):



        # interpolate 2D data onto grid
        print(len(invar))
        extent, true_outvar, pred_outvar = interpolate_2D(
                200, invar, true_outvar, pred_outvar
            )
        ndim=2
        # make plots
        dims = list(invar.keys())
        fs = []
        for k in pred_outvar:
            f = plt.figure(figsize=(3 * 5, 4), dpi=100)
            for i, (o, tag) in enumerate(
                zip(
                    [true_outvar[k], pred_outvar[k], ((true_outvar[k] - pred_outvar[k])**2)**(0.5) ],
                    ["true", "pred", "diff"],
                )
            ):
                if (tag=="diff"):
                    print("max diff",np.max(o))
                plt.subplot(1, 3, 1 + i)
                if ndim == 1:
                    plt.plot(invar[dims[0]][:, 0], o[:, 0])
                    plt.xlabel(dims[0])
                elif ndim == 2:
                    plt.imshow(o.T, origin="lower", extent=extent)
                    
                    plt.xlabel(dims[0])
                    plt.ylabel(dims[1])
                    if(tag=="diff"):
                       
                        plt.clim(0,0.6)
                    else :
                        plt.clim(0,1)
                    plt.colorbar()
                plt.title(f"{k}_{tag}")
            plt.tight_layout()
            fs.append((f, k))
     
def plot(base_dir = "outputs/fhnw/window0/validators/"):
    

    data = np.load(base_dir, allow_pickle=True)
    data = np.atleast_1d(data.f.arr_0)[0]
    invar={
           "t":data["t"],"K":data["K"]*15
          }
    print(len(invar))
    out={"x1":data["pred_x1"],
         }
    out_t={"x1":data["true_x1"],
          }
    
    print (np.shape(out_t["x1"]))
    call(invar,out_t,out)
    print(len(invar))
    out={"w":data["pred_w"],
         }
    out_t={"w":data["true_w"],
          }
    
    print (np.shape(out_t["w"]))
    call(invar,out_t,out)
    
    
plot(base_dir = "outputs/fhn2eqtw/window_0009/validators/validator.vtp")


FileNotFoundError: [Errno 2] No such file or directory: 'outputs/fhn2eqtw/window_0009/validators/validator.vtp'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def find(list_to_check, item_to_find):
    return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]

base_dir =  "outputs/fhn2eqtw/window1/validators/"

# plot in 1d
data = np.load(base_dir + "validator.npz", allow_pickle=True)
data = np.atleast_1d(data.f.arr_0)[0]

ks=np.unique(data["K"])
t=np.unique(data["t"])
d=np.full_like(ks,0)


for k in ks:
    i=find(data["K"],k)
    #print(i)
    x=data["true_x1"][i]
    pred=data["pred_x1"][i]
    w=data["true_w"][i]
    predw=data["pred_w"][i]
    d=np.mean(x-pred)
    plt.ylim(-1,1.1)
    plt.plot(t,x,"o",label="True ,k ="+ str(k))
    plt.plot(t,pred,label="Pred ,k ="+ str(k))
    plt.plot(t,w,"o",label="True w ,k ="+ str(k))
    print(predw[-1])
    plt.plot(t,predw,label="Pred w ,k ="+ str(k))
   
    plt.legend(loc="best")
    plt.show()